In [4]:
#Lam Ho Wang SID: 55678242
import Crypto 
import Crypto.Random
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5 
from Crypto.Hash import SHA256
import binascii
import json
import datetime

In [10]:
class Transaction:
    def __init__(self, sender,recipient,value):
        self.sender = sender
        self.recipient = recipient
        self.value = value
    def to_dict(self):
        return ({'sender': self.sender , 'recipient': self.recipient, 'value': self.value})

    def add_signature(self, signature_):
        self.signature = signature_
    
    def verify_transaction_signature(self):
        if hasattr(self, 'signature'):
            public_key = RSA.importKey(binascii.unhexlify(self.sender))
            verifier = PKCS1_v1_5.new(public_key)
            h = SHA256.new(str(self.to_dict()).encode('utf8'))
            return verifier.verify(h, binascii.unhexlify(self.signature))
        else:
            return False

    def to_json(self):
        return json.dumps( self.__dict__, sort_keys = False)


In [11]:
class Wallet:
    def __init__(self):
        random = Crypto.Random.new().read
        self._private_key = RSA.generate(1024,random)
        self._public_key = self._private_key.publickey()
    
    def sign_transaction(self, transaction: Transaction) :
        signer = PKCS1_v1_5.new(self._private_key)
        h = SHA256.new(str(transaction.to_dict()).encode('utf8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')


    @property
    def identity(self):
        pubkey = binascii.hexlify( self._public_key.exportKey(format='DER'))
        return pubkey.decode('ascii')

    @property
    def private(self):
        private_key = binascii.hexlify( self._private_key.exportKey(format='DER'))
        return private_key.decode('ascii')

In [25]:
class Block :
    def __init__(self, index, transactions, timestamp, previous_hash):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.hash = None
        self.nonce = 0
    def to_dict(self):
        return ({
            'index' : self.index,
            'transactions' : self.transactions,
            'timestamp' : self.timestamp,
            'previous_hash' : self.previous_hash,
            'nonce' : self.nonce}
        )
    def to_json(self):
        return json.dumps (self.__dict__)
    def compute_hash( self):
        return SHA256.new(str(self.to_dict()).encode()).hexdigest()

In [26]:
class Blockchain:
    difficulty = 2

    def __init__( self):
        self.unconfirmed_transactions = []
        self.chain = []
        self.create_genesis_block()

    def create_genesis_block( self):
        genesis_block = Block(0, [], datetime.datetime.now()
                                .strftime("%m/%d/%Y, %H:%M:%S"),"0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block.to_json())
    def add_new_transaction(self, transaction: Transaction):
        if transaction.verify_transaction_signature():
            self.unconfirmed_transactions.append(transaction.to_json())
            return True
        else:
            return False

    def add_block( self, block, proof):

        previous_hash = self.last_block[ 'hash' ]

        if previous_hash != block.previous_hash:
            return False
        
        if not self.is_valid_proof(block, proof):
            return False
        
        block.hash = proof
        self.chain.append(block.to_json())
        return True
    def is_valid_proof(self, block, block_hash):

        return (block_hash.startswith( '0' * Blockchain.difficulty) and
                block_hash == block.compute_hash())
    def proof_of_work(self, block ):
        block.nonce = 0
        computed_hash = block.compute_hash()
        while not computed_hash.startswith( '0' * Blockchain.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()
        return computed_hash
    def mine(self, mywallet) :
        block_reward = Transaction( "Block_Reward" ,
                                    mywallet.identity, "5.0" ).to_json()
        self.unconfirmed_transactions.insert(0, block_reward)
        if not self.unconfirmed_transactions:
            return False

        new_block = Block(index=self.last_block['index'] + 1,
                            transactions=self.unconfirmed_transactions,
                            timestamp=datetime.datetime.now()
                            .strftime("%m/%d/%Y, %H:%M:%S"),
                            previous_hash=self.last_block['hash'])

        proof = self.proof_of_work(new_block)
        if self.add_block(new_block, proof):
            self.unconfirmed_transactions = []
            return new_block
        else:
            return False
    @property
    def last_block(self):
        return json.loads(self.chain[-1])

In [27]:
blockchain = Blockchain()
Peter = Wallet()
Mary = Wallet()

t = Transaction(Peter.identity,Mary.identity,250)
t.add_signature(Peter.sign_transaction(t))
blockchain.add_new_transaction(t)

blockchain.mine(Peter)
print(blockchain.last_block)
blockchain.mine(Peter)
print(blockchain.last_block)


{'index': 1, 'transactions': ['{"sender": "Block_Reward", "recipient": "30819f300d06092a864886f70d010101050003818d0030818902818100d6990148a2eb102986af0ed2795fff0c900861555550de0e25adaea2613b086926c2d4bfc50b35afb66820f37fcc38a50aebf38973285c478af621749b37f1582c78bf91951a28fa599a5c741a8daaf5d0dd9107062466def4250838572b575c23e01d5591a1f869cce501d73885172bb9335b093fdec342796d889f7e2943d90203010001", "value": "5.0"}', '{"sender": "30819f300d06092a864886f70d010101050003818d0030818902818100d6990148a2eb102986af0ed2795fff0c900861555550de0e25adaea2613b086926c2d4bfc50b35afb66820f37fcc38a50aebf38973285c478af621749b37f1582c78bf91951a28fa599a5c741a8daaf5d0dd9107062466def4250838572b575c23e01d5591a1f869cce501d73885172bb9335b093fdec342796d889f7e2943d90203010001", "recipient": "30819f300d06092a864886f70d010101050003818d0030818902818100bdd4942d832e3d6f96634d2d040aa846b05a176ea9d15e773a52f8fe039c0aa0c981b4575349f81f681a486e990c7080617703190b254132efc1d2e7e5d060ab1da4c8498300a2a75e0eedcf91319bd882146054b32